In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import Helper_functions as hf

In [2]:
(train_images, train_labels), (validation_images, validation_labels) = tf.keras.datasets.mnist.load_data()
#train_images = np.expand_dims(train_images, axis = 3)
#validation_images = np.expand_dims(validation_images, axis = 3)
print(train_images.shape, train_labels.shape)

(60000, 28, 28) (60000,)


60k 28x28 train images with labels  
10k 28x28 validation images with labels  
now build the mnist model

In [67]:
def change_set_statistics(images: np.ndarray, labels: np.ndarray, mu, rho):
    #check tensor dimensions
    if images.shape[0] != labels.shape[0]:
        raise IndexError
    array_dict = dict()
    #set majority class size to the minimal avaliable class size
    majority_size = np.min(np.unique(labels, return_counts=True)[1])
    #extract every class from the images array, make it an array, shuffle it, 
    #and shorten in length such that len(every class) == majority_size
    for i in np.unique(labels):
        array_dict[i] = train_images[np.where(train_labels == i)]
        np.random.shuffle(array_dict[i])
        array_dict[i] = array_dict[i][:majority_size]
    #set minority class size
    minority_size = int(np.round(majority_size / rho))
    #if step imbalance variant chosen
    if mu is not None:
        #set numbers of minority and majroity classes
        minority_no = int(np.round(mu * len(array_dict)))
        majority_no = len(array_dict) - minority_no
        #create list of unique shuffled labels
        list_of_labels = np.unique(labels)
        np.random.shuffle(list_of_labels)
        #init output np arrays
        out_labels = np.empty(shape = tuple(labels.shape[i] if i != 0
                                            else 0
                                            for i in range(len(labels.shape))))
        out_images = np.empty(shape = tuple(images.shape[i] if i != 0 
                                            else 0
                                            for i in range(len(images.shape))))
        #add minority classes to concatenaded output tensors
        for i in range(minority_no):
            array_dict[list_of_labels[i]] = array_dict[list_of_labels[i]][:minority_size]
            tmp_labels = np.full(shape = tuple(labels.shape[i] if i != 0
                                            else minority_size
                                            for i in range(len(labels.shape))), 
                                 fill_value = list_of_labels[i])
            out_images = np.concatenate((out_images, array_dict[list_of_labels[i]]))
            out_labels = np.concatenate((out_labels, tmp_labels))
        for i in range(minority_no, len(array_dict), 1):
            tmp_labels = np.full(shape = tuple(labels.shape[i] if i != 0
                                            else majority_size
                                            for i in range(len(labels.shape))), 
                                 fill_value = list_of_labels[i])
            out_images = np.concatenate((out_images, array_dict[list_of_labels[i]]))
            out_labels = np.concatenate((out_labels, tmp_labels))
        return (out_images, out_labels)
    #todo: mu is None    

In [69]:
(obrazki, naklejki) = change_set_statistics(train_images, train_labels, 0.2, 2)
print(hf.Imbalance.get_set_distribution(naklejki))

{0.0: 2710, 1.0: 5421, 2.0: 5421, 3.0: 5421, 4.0: 5421, 5.0: 5421, 6.0: 5421, 7.0: 2710, 8.0: 5421, 9.0: 5421}


In [20]:
zeros = train_images[np.where(train_labels == 0)]
print(len(train_labels.shape))
print(tuple(train_labels.shape[i] for i in range(len(train_labels.shape))))

1
(60000,)


In [6]:
print(hf.Imbalance.get_set_distribution(train_labels))

{0: 5923, 1: 6742, 2: 5958, 3: 6131, 4: 5842, 5: 5421, 6: 5918, 7: 6265, 8: 5851, 9: 5949}


In [56]:
out_labels = np.empty(shape = tuple(train_labels.shape[i] if i != 0
                                            else 0
                                            for i in range(len(train_labels.shape))))
np.concatenate(out_labels, np.full(shape = 100, fill_value = 1))

TypeError: only integer scalar arrays can be converted to a scalar index